# [Manifold tutorial](https://nbviewer.org/github/sagemanifolds/SageManifolds/blob/master/Notebooks/SM_tutorial.ipynb) from SageManifolds Tutorial

In [1]:
version()
print(SAGE_ROOT)

/home/ernest/TQFT/sage


Set up the notebook to display mathematical objects using LaTeX rendering.

In [2]:
%display latex

## Defining a manifold
Define a differentiable manifold of dimension 3 over $\mathbb{R}$.

In [3]:
M = Manifold(3, 'M', latex_name=r'\mathcal{M}', start_index=1)

In [4]:
print(M)
print(type(M))
M

3-dimensional differentiable manifold M
<class 'sage.manifolds.differentiable.manifold.DifferentiableManifold_with_category'>


3-dimensional differentiable manifold M

We can ask for the category of M and see that it's the category of smooth manifolds over $\mathbb{R}$.

In [5]:
category(M)

Category of smooth manifolds over Real Field with 53 bits of precision

The indices on the manifold are generated by method `irange()`, used in loops.

In [6]:
[i for i in M.irange()]

[1, 2, 3]

## Defining a chart on the manifold

`chart()` in this example has no arguments, which implies coordinate symbols will be `x, y, z` and each coordinate range is `(-oo, +oo)`; for other cases, an argument must be passed to `chart()` to specify the coordinate symbols and range.

In [7]:
X.<x, y, z> = M.chart()

In [8]:
print(X)
X

Chart (M, (x, y, z))


Chart (M, (x, y, z))

In [9]:
X[1]

x

In [10]:
X[2]

y

In [11]:
X[3]

z

In [12]:
X[:]

(x, y, z)

In [14]:
print(type(z))
z is X[3]

<class 'sage.symbolic.expression.Expression'>


True

## Functions of the chart coordinates

Real-valued functions of the chart coordinates are generated via method `function()`, acting on the chart:

In [18]:
f = X.function(x + y^2 + z^3)
f

<class 'sage.manifolds.chart_func.ChartFunctionRing_with_category.element_class'>


z^3 + y^2 + x

In [16]:
f.display()

(x, y, z) ↦ z^3 + y^2 + x

In [17]:
f(1,2,3)

32

In [19]:
print(type(f))

<class 'sage.manifolds.chart_func.ChartFunctionRing_with_category.element_class'>


class `ChartFunction` differ from SageMath standard symbolic functions by automatic simplifications in all operations. For instance,

In [20]:
f0(x,y,z) = cos(x)^2; g0(x,y,z) = sin(x)^2
f0 + g0

(x, y, z) |--> cos(x)^2 + sin(x)^2

In [21]:
f1 = X.function(cos(x)^2); g1 = X.function(sin(x)^2)
f1 + g1

1

In [22]:
(f0 + g0).simplify_trig()

(x, y, z) |--> 1

In [26]:
print(f0)
print(f1.display())
print(f1.expr())
print(type(f1.expr()))
f1

(x, y, z) |--> cos(x)^2
(x, y, z) ↦ cos(x)^2
cos(x)^2
<class 'sage.symbolic.expression.Expression'>


cos(x)^2

## Introducing a second chart on the manifold

Let us first consider an open subset of $M$, for instance the complement $U$ of the region defined by $\lbrace y = 0, x \geq 0 \rbrace$ (note that `(y!=0, x<0)` stands for $y\neq 0$ OR $x < 0$; the condition $y\neq 0$ AND $x< 0$ would have been written `[y!=0, x<0]` instead):

In [27]:
U = M.open_subset('U', coord_def={X: (y!=0, x<0)})

Let us call `X_U` the restriction of the chart `X` to open subset $U$:

In [28]:
X_U = X.restrict(U)
X_U

Chart (U, (x, y, z))

Introduce another chart on U with spherical coordinates $(r, \theta, \phi)$.

EY: I found that the code doesn't result in error if the choice of variable name on the LHS, `r, th, ph` do not match exactly the "string" name for the coordinate, "r, th, ph". With respect to the "code" or what Python sees, it's the LHS that gives the actual variable name to call.

In [49]:
Y.<r,th,ph> = U.chart(r'r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
Y

Chart (U, (r, th, ph))

In [38]:
th, ph

(theta1, phi42)

In [39]:
Y[2], Y[3]

(theta1, phi42)

In [40]:
assumptions()

[x is real,
 y is real,
 z is real,
 r is real,
 r > 0,
 th is real,
 th > 0,
 th < pi,
 ph is real,
 ph > 0,
 ph < 2*pi,
 radius is real,
 radius > 0,
 theta is real,
 theta > 0,
 theta < pi,
 phi is real,
 phi > 0,
 phi < 2*pi,
 theta1 is real,
 theta1 > 0,
 theta1 < pi,
 phi42 is real,
 phi42 > 0,
 phi42 < 2*pi]

In [45]:
print(simplify(abs(r)))
simplify(abs(x)) # no simplification occurs since x can take any value in R

r


abs(x)

In [44]:
simplify(abs(x))

abs(x)

After having been declared, the chart Y can be fully specified by its relation to the chart X_U, via a transition map:

In [50]:
transit_Y_to_X = Y.transition_map(X_U, [r*sin(th)*cos(ph), r*sin(th)*sin(ph),r*cos(th)])
transit_Y_to_X

Change of coordinates from Chart (U, (r, th, ph)) to Chart (U, (x, y, z))

In [51]:
transit_Y_to_X.display()

x = r*cos(ph)*sin(th)
y = r*sin(ph)*sin(th)
z = r*cos(th)

The inverse of the transition map can be specified by means of the method `set_inverse()`.

A check of the provided inverse is performed by composing it with the original transition map.

In [52]:
transit_Y_to_X.set_inverse(sqrt(x^2+y^2+z^2), atan2(sqrt(x^2+y^2),z), atan2(y, x))

Check of the inverse coordinate transformation:
  r == r  *passed*
  th == arctan2(r*sin(th), r*cos(th))  **failed**
  ph == arctan2(r*sin(ph)*sin(th), r*cos(ph)*sin(th))  **failed**
  x == x  *passed*
  y == y  *passed*
  z == z  *passed*
NB: a failed report can reflect a mere lack of simplification.


In [54]:
transit_Y_to_X.inverse().display()

r = sqrt(x^2 + y^2 + z^2)
th = arctan2(sqrt(x^2 + y^2), z)
ph = arctan2(y, x)

In [53]:
M.atlas()

[Chart (M, (x, y, z)),
 Chart (U, (x, y, z)),
 Chart (U, (r, th, ph)),
 Chart (U, (radius, theta, ph)),
 Chart (U, (r, theta, phi)),
 Chart (U, (r, theta1, phi42))]

In [55]:
M.default_chart()

Chart (M, (x, y, z))

Each open subset has its own atlas (since an open subset of a manifold is a manifold by itself).

In [56]:
U.atlas()

[Chart (U, (x, y, z)),
 Chart (U, (r, th, ph)),
 Chart (U, (radius, theta, ph)),
 Chart (U, (r, theta, phi)),
 Chart (U, (r, theta1, phi42))]

In [57]:
U.default_chart()

Chart (U, (x, y, z))

We can draw the chart Y in terms of the chart X via command `Y.plot(X)`, which shows the lines of constant coordinates from the Y chart in a "Cartesian frame" based on the X coordinates:

In [58]:
Y.plot(X)

Graphics3d Object

In [59]:
Y.plot(X, ranges={r:(1,2), th:(0,pi/2)}, number_values=4,
      color={r:'blue', th:'green', ph:'red'}, aspect_ratio=1)

Graphics3d Object

Conversely, the chart $\left. X \right|_U$ can be plotted in terms of the chart $Y$ (this isn't possible for the whole chart $X$ since its domain is larger than that of chart $Y$)

In [60]:
graph = X_U.plot(Y)
show(graph, axes_labels=['r', 'theta', 'phi'])

Graphics3d Object

## Points on the manifold

A point on $M$ is defined by its coordinates in a given chart:

In [61]:
p = M.point((1,2,-1), chart=X, name='p')
print(p)
p

Point p on the 3-dimensional differentiable manifold M


Point p on the 3-dimensional differentiable manifold M

In [62]:
print(p in M)
print(p in U)
print(p.coord(X))
p.coord()

True
True
(1, 2, -1)


(1, 2, -1)

In [63]:
X(p)

(1, 2, -1)

In [64]:
q = M.point((1,0,2), name='q')
print(q in U)
try:
    q.coord(Y)
except ValueError as exc:
    print("Error: " + str(exc))

False
Error: the point does not belong to the domain of Chart (U, (r, th, ph))
